<a href="https://colab.research.google.com/github/kimhwijin/TensorflowWithKeras/blob/master/GAN/Age_cGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import time
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras  import Input, Model, layers, optimizers, utils
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.callbacks import TensorBoard
from keras_preprocessing import image
from scipy.io import loadmat
import os
from datetime import datetime

In [4]:
def load_data(wiki_dir, dataset='wiki'):
  meta = loadmat(os.path.join(wiki_dir, "{}.mat".format(dataset)))
  full_path = meta[dataset][0, 0]["full_path"][0]
  dob = meta[dataset][0, 0]["dob"][0]
  photo_taken = meta[dataset][0, 0]["photo_taken"][0]
  age = [calculate_age(photo_taken[i], dob[i]) for i in range(len(dob))]
  images = []
  age_list = []

  for index, image_path in enumerate(full_path):
    images.append(image_path[0])
    age_list.append(age[index])

  return images, age_list

def caluclate_age(taken, dob):
  birth = datetime.fromordinal(max(int(dob) - 366, 1))
  if brith.month < 7:
    return taken - brith.year
  else:
    return taken - birth.year - 1
os.path.isfile("drive/MyDrive/wiki_crop/wiki.mat")
#images, age_list = load_data("drive/MyDrive/wiki_crop/", dataset='wiki')

False

In [ ]:
def build_encoder():
  input_layer = Input(shape=(64,64,3))
  enc = layers.Conv2D(filters=32, kernel_size=5, strides=2, padding='same')(input_layer)
  enc = layers.LeakyReLU(alpha=0.2)(enc)

  enc = layers.Conv2D(filters=64, kernel_size=5, strides=2, padding='same')(enc)
  enc = layers.BatchNormalization()(enc)
  enc = layers.LeakyReLU(alpha=0.2)(enc)

  enc = Conv2D(filters=128, kernel_size=5, strides=2, padding='same')(enc)
  enc = layers.BatchNormalization()(enc)
  enc = layers.LeakyReLU(alpha=0.2)(enc)

  enc = Conv2D(filters=256, kernel_size=5, strides=2, padding='same')(enc)
  enc = layers.BatchNormalization()(enc)
  enc = layers.LeakyReLU(alpha=0.2)(enc)

  enc = layers.Flatten()(enc)

  enc = layers.Dense(4096)(enc)
  enc = layers.BatchNormalization()(enc)
  enc = layers.LeakyReLU(alpha=0.2)(enc)

  enc = layers.Dense(100)(enc)
  
  model = Model(inputs=[input_layer], outputs=[enc])
  
  return model


In [ ]:

def build_generator():
  latent_dims = 100
  num_classes = 6

  input_z_noise = Input(shape=(latent_dims,))
  input_label = Input(shape=(num_classes,))

  x = layers.concatenate([input_z_noise, input_label])

  x = layers.Dense(2048, input_dim=latent_dims+num_classes)(x)
  x = layers.LeakyReLU(alpha=0.2)(x)
  x = Dropout(0.2)(x)

  x = layers.Dense(256 * 8 * 8)(x)
  x = layers.BatchNormalization(momentum=0.8)(x)
  x = layers.LeakyReLU(alpha=0.2)(x)
  x = Dropout(0.2)(x)

  x = layers.Reshape(8, 8, 256)(x)

  x = layers.UpSampling2D(size=(2, 2))(x)
  x = layers.Conv2D(filters=128, kernel_size=5, padding='same')(x)
  x = layers.BatchNormalization(momentum=0.8)(x)
  x = layers.LeakyReLU(alpha=0.2)(x)

  x = layers.UpSampling2D(size=(2, 2))(x)
  x = layers.Conv2D(filters=64, kernel_size=5, padding='same')(x)
  x = layers.BatchNormalization(momentum=0.8)(x)
  x = layers.LeakyReLU(alpha=0.2)(x)

  x = layers.UpSampling2D(size=(2, 2))(x)
  x = layers.Conv2D(filters=3, kernel_size=5, padding='same')(x)
  x = layers.Activation('tahn')(x)

  model = Model(inputs=[input_z_noise, input_label], output=[x])
  return model

In [ ]:
def build_discriminator():
  input_shape = (64, 64, 3)
  label_shape = (6, )

  image_input = layers.Input(shape=input_shape)
  label_input = layers.Input(shape=label_shape)

  x = layers.Conv2D(filters=64, kernel_size=3, strides=2, pading='same')(image_input)
  x = layers.LeakyReLU(alpha=0.2)(x)

  label_input1 = layers.Lambda(expand_label_input)(label_input)
  def expand_label_input(x):
    x = K.expand_dims(x, axis=1)
    x = K.expand_dims(x, axis=1)
    x = K.tile(x, [1, 32, 32, 1])
    return x
  
  x = layers.concatenate([x, label_input1], axis=3)

  x = layers.Conv2D(128, kernel_size=3, strides=2, padding='same')(x)
  x = layers.BatchNormalization()(x)
  x = layers.LeakyReLU(alpha=0.2)(x)

  x = layers.Conv2D(256, kernel_size=3, strides=2, padding='same')(x)
  x = layers.BatchNormalization()(x)
  x = layers.LeakyReLU(alpha=0.2)(x)

  x = layers.Conv2D(512, kernel_size=3, strides=2, padding='same')(x)
  x = layers.BatchNormalization()(x)
  x = layers.LeakyReLU(alpha=0.2)(x)

  x = layers.Flatten()(x)

  x = layers.Dense(1, activation='sigmoid')(x)

  model = Model(inputs=[image_input, label_input], outputs=[x])
  return model

In [ ]:
data_dir = "drive/MyDrive/Datasets/"
wiki_dir = os.path.join(data_dir, "wiki_crop")
epochs = 500
batch_size = 128
image_shape = (64, 64, 3)
z_shape = 100
TRAIN_GAN = True
TRAIN_ENCODER = False
TRAIN_GAN_WITH_FR = False
fr_image_shape = (192, 192, 3)

dis_optimizer = optimizers.Adam(lr=0.0002, beta_1=0.5, beta_2=0.999, epsilon=10e-8)
gen_optimizer = optimizers.Adam(lr=0.0002, beta_1=0.5, beta_2=0.999, epsilon=10e-8)
adversarial_optimizer = optimizers.Adam(lr=0.0002, beta_1=0.5, beta_2=0.999, epsilon=10e-8)

discriminator = build_discriminator()
discriminator.compile(loss=['binary_crossentropy'], optimizers=dis_optimizer)

generator = build_generator()
generator.compile(loss=['binary_crossentropy'], optimizers=gen_optimizer)

discriminator.trainable = False
input_z_noise = Input(shape=(z_shape,))
input_label = Input(shape=(6,)
reconstruct_images = generator([input_z_noise, input_label])
valid = discriminator([reconstruct_images, input_label])
adversarial_model = Model(inputs=[input_z_noise, input_label])
adversarial_model.compile(loss=['binary_crossentropy'], optimizer=gen_optimizer)


In [ ]:
tensorboard = TensorBoard(log_dir='drive/MyDrive/Colab Notebooks/logs/{}'.format(time.time()))
tensorboard.set_model(generator)
tensorboard.set_model(discriminator)

images, age_list = load_data(wiki_dir=wiki_dir, dataset="wiki")
age_category = age_to_category(age_list)
def age_to_category(age_list):
  age_list1 = []
  for age in age_list:
    if 0 < age <= 18:
      age_category = 0
    elif 18 < age <= 29:
      age_category = 1
    elif 29 < age <= 39:
      age_category = 2
    elif 39 < age <= 49:
      age_category = 3
    elif 49 < age <= 59:
      age_category = 4
    elif 60 < age:
      age_category = 5
    age_list1.append(age_category)

  return age_list1

final_age_category = np.reshape(np.array(age_category), [len(age_category), 1])
classes = 6
y = utils.to_categorical(final_age_category, num_classes=classes)

In [ ]:
def load_images(data_dir, image_paths, image_shape):
  images = None
  for i , image_path in enumerate(image_paths):
    print()
    try:
      loaded_image = image.load_img(os.path.join(data_dir, image_path), target_size=image_shape)
      loaded_image = image.img_to_array(loaded_image)
      loaded_image = np.expand_dims(loaded_image, axis=0)

      if images is None:
        images = loaded_image
      else:
        images = np.concatenate([images, loaded_image], axis=0)
    except Exception as e:
      print("Error: ", i, e)
  return images

loaded_images = load_images(wiki_dir, images, (images_shape[0], images_shape[1]))

In [ ]:
for epoch in range(epochs):
  print("Epoch : {}".format(epoch))
  gen_losses = []
  dis_losses = []

  number_of_batches = int(len(loaded_images) / batch_size)
  print("Number of batches : " ,number_of_batches)

  for index in range(number_of_batches):
    print("Batch: {}".format(index + 1))
    images_batch = loaded_image[index * batch_size :(index + 1) * batch_size]
    images_batch = images_batch / 127.5 - 1.0 #-1 to 1
    images_batch = images_batch.astype(np.float32)
    y_batch = y[index * batch_size : (index + 1) * batch_size]

    z_noise = np.random.normal(0, 1, size=(batch_size, z_shape))
    #이미지 생성
    initial_reconstruct_images = generator.predict_on_batch([z_noise, y_batch])

    #판별기 훈련
    d_loss_real = discriminator.train_on_batch([images_batch, y_batch], real_labels)
    d_loss_fake = discriminator.train_on_batch([initial_reconstruct_images, y_batch], fake_labels)

    #GAN 훈련
    z_noise2 = np.random.normal(0, 1, size=(batch_size, z_shape))
    ramdon_labels = np.random.randint(0, 6, batch_size).reshape(1, -1)
    random_labels = utils.to_categorical(random_labels, 6)

    g_loss = adversarial_model.train_on_batch([z_noise2, random_labels], [1] * batch_size)

    d_loss = 0.5 * np.add(d_loss_read, d_loss_fake)
    print("d_loss: ", d_loss)
    print("g_loss: ", g_loss)

    gen_losses.append(g_loss)
    dis_losses.append(d_loss)

  write_log(tensorboard, 'g_loss', np.mean(gen_losses), epoch)
  write_log(tensorboard, 'd_loss', np.mean(dis_losses), epoch)

  if epoch % 10 == 1:
    for i , img in enumerate(initial_reconstruct_images):
      save_rgb_img(img, path="drive/MyDrive/Colab Notebooks/result_data/wiki_gan/img_{}_{}.png".format(epoch,i))
  
generator.save_weight("drive/MyDrive/Colab Notebooks/models/wiki_gan/weight/generator.h5")
discriminator.save_weight("drive/MyDrive/Colab Notebooks/models/wiki_gan/weight/discriminator.h5")

generator.save("drive/MyDrive/Colab Notebooks/models/wiki_gan/model/generator.h5")
discriminator.save("drive/MyDrive/Colab Notebooks/models/wiki_gan/model/discriminator.h5")


In [ ]:
def save_rgb_img(img, path):
  fig = plt.figure()
  ax = fig.add_subplot(1,1,1)
  ax.imshow(img)
  ax.axis("off")
  ax.set_title("Image")
  plt.savefig(path)
  plt.close()


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
